In [ ]:
import cv2 as cv
import pytesseract
import tensorflow as tf
import re

In [ ]:
img = cv.imread('./datasets/products/20240515_161317.png')

In [ ]:
dataset = tf.data.TFRecordDataset(["datasets/products/labels/test/nutrition-label.tfrecord", "datasets/products/labels/train/nutrition-label.tfrecord"])

# Bounding boxes for nutrition label on submitted photo

In [315]:
featureNames = {
    "filename": "image/filename",
    "format": "image/format",
    "ymin": "image/object/bbox/ymin",
    "ymax": "image/object/bbox/ymax",
    "xmax": "image/object/bbox/xmax",
    "xmin": "image/object/bbox/xmin"
}

dataset_path = "./datasets/products/labels/train/nutrition-label.tfrecord"

# Define the feature description
feature_description = {
    featureNames["filename"]: tf.io.FixedLenFeature([], tf.string),
    featureNames["format"]: tf.io.FixedLenFeature([], tf.string),
    featureNames["xmin"]: tf.io.VarLenFeature(tf.float32),
    featureNames["xmax"]: tf.io.VarLenFeature(tf.float32),
    featureNames["ymin"]: tf.io.VarLenFeature(tf.float32),
    featureNames["ymax"]: tf.io.VarLenFeature(tf.float32),
}

def _parse_image_function(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    filename = parsed_features[featureNames["filename"]]
    
    # TODO: this line produced an error because filename is in the format 'SymbolicTensor', yet regex requires it to be a string
    # 1. figure out how to either convert 'SymbolicTensor' to string, or
    # 2. get another way to get value for 'image/filename' feature in the .tfrecord file
    cleaned_filename = re.sub(r'\.rf\.[^.]+\.jpg', '', filename)
    
    result_dict = {
        "file": tf.strings.join([cleaned_filename,parsed_features[featureNames["format"]]],separator="."),
        "bb": {
            "xmin": tf.sparse.to_dense(parsed_features[featureNames["xmin"]])[0],
            "xmax": tf.sparse.to_dense(parsed_features[featureNames["xmax"]])[0],
            "ymin": tf.sparse.to_dense(parsed_features[featureNames["ymin"]])[0],
            "ymax": tf.sparse.to_dense(parsed_features[featureNames["ymax"]])[0],
        }
    }
    
    return result_dict


raw_image_dataset = tf.data.TFRecordDataset(dataset_path)
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)


TypeError: in user code:

    File "/var/folders/0_/z1kw8mhx7gb7gqt3tft89bdm0000gn/T/ipykernel_59192/234655366.py", line 28, in _parse_image_function  *
        cleaned_filename = re.sub(r'\.rf\.[^.]+\.jpg', '', filename)
    File "/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12/re/__init__.py", line 186, in sub  **
        return _compile(pattern, flags).sub(repl, string, count)

    TypeError: expected string or bytes-like object, got 'SymbolicTensor'


In [ ]:
for item in parsed_image_dataset.take(1):
    bounding_boxes = item["bb"]
    filename = item["file"]

    print(f"Filename: {filename}")
    print(bounding_boxes['xmin'])